In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import h5py
import cv2
import os
import glob
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Reshape, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adadelta, Adam
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, History
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from __future__ import division, print_function
from collections import Counter
%matplotlib inline

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]

def VGG_16(size=(224, 224), weights_path='data/vgg16_bn.txt'):
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,)+size))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [3]:
size = (224, 224)
model = VGG_16(size=size)
model.pop(); model.pop(); model.pop(); model.pop(); model.pop();

for layer in model.layers:
    layer.trainable = False

/usr/local/lib/python2.7/dist-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [6]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
            'data/train_org_gen/',
            target_size=size,
            batch_size=16,
            shuffle = False,
            seed = np.random.random_integers(0, 100000),
            classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'],
            class_mode = 'categorical')

Found 3777 images belonging to 8 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: DeprecationWarning: This function is deprecated. Please call randint(0, 100000 + 1) instead


In [7]:
test_generator.filenames

['ALB/img_06622.jpg',
 'ALB/img_06518.jpg',
 'ALB/img_00351.jpg',
 'ALB/img_04877.jpg',
 'ALB/img_05812.jpg',
 'ALB/img_01744.jpg',
 'ALB/img_02931.jpg',
 'ALB/img_01139.jpg',
 'ALB/img_06720.jpg',
 'ALB/img_03579.jpg',
 'ALB/img_03304.jpg',
 'ALB/img_01506.jpg',
 'ALB/img_07418.jpg',
 'ALB/img_05884.jpg',
 'ALB/img_07811.jpg',
 'ALB/img_04778.jpg',
 'ALB/img_03427.jpg',
 'ALB/img_07704.jpg',
 'ALB/img_07698.jpg',
 'ALB/img_05270.jpg',
 'ALB/img_04552.jpg',
 'ALB/img_01460.jpg',
 'ALB/img_00596.jpg',
 'ALB/img_07903.jpg',
 'ALB/img_05725.jpg',
 'ALB/img_05845.jpg',
 'ALB/img_06629.jpg',
 'ALB/img_05022.jpg',
 'ALB/img_07549.jpg',
 'ALB/img_07807.jpg',
 'ALB/img_03985.jpg',
 'ALB/img_00233.jpg',
 'ALB/img_00899.jpg',
 'ALB/img_02519.jpg',
 'ALB/img_07789.jpg',
 'ALB/img_04659.jpg',
 'ALB/img_07878.jpg',
 'ALB/img_03005.jpg',
 'ALB/img_04848.jpg',
 'ALB/img_00482.jpg',
 'ALB/img_05570.jpg',
 'ALB/img_06907.jpg',
 'ALB/img_06950.jpg',
 'ALB/img_06553.jpg',
 'ALB/img_02283.jpg',
 'ALB/img_

In [8]:
predictions = model.predict_generator(test_generator, 3777)

In [9]:
predictions

array([[ -13.53399658,  -10.28162956,   -6.40039444, ...,   -5.1003952 ,
          36.20607758,    2.72901726],
       [ -13.53399658,  -10.28162956,   -6.40039444, ...,   -5.1003952 ,
          28.69327736,   -5.22272015],
       [ -13.53399658,  -10.28162956,   -6.40039444, ...,   -5.1003952 ,
          45.88959885,   -5.22272015],
       ..., 
       [  35.84438705,  -10.28162956,   -6.40039444, ...,   -5.1003952 ,
          14.69765663,   -5.22272015],
       [ -13.53399658,   14.54412842,   -6.40039444, ...,   -5.1003952 ,
         105.56240082,   -5.22272015],
       [ -13.53399658,  -10.28162956,   -6.40039444, ...,   -5.1003952 ,
          60.05918503,   -5.22272015]], dtype=float32)

In [10]:
predictions.shape

(3777, 4096)